# BIDS変換プロトコル
目的：根本清貴先生のbatch-heudiconvをお借りして、抽出したDICOMファイルをBIDS形式に変換するフローを作成する
https://github.com/kytk/batch-heudiconv.git

必要なソフトウェア
・dcm2niix
・pydicom
・heudiconv

In [1]:
#Parameters
project = "NIIDtest"
pattern = "{subject}"
wd = "/Volumes/YN_Research"
path = "/Users/yn_mbp/git/yn-batch-heudiconv"

## ディレクトリの準備

In [2]:
import os

#working directoryへ移動
os.chdir(wd)

In [3]:
#ディレクトリ構造の準備
!bh01_prep_dir.sh {project}

Directory structure for NIIDtest has been prepared:
- DICOM/original/  : Place your original DICOM files here
- DICOM/sorted/    : Sorted DICOM files will be stored here
- DICOM/converted/ : Backup of processed DICOM files
- bids/           : BIDS-formatted output will be stored here
- code/           : Location for heuristic files
- tmp/            : Temporary files

Please copy DICOM files to DICOM/original/ directory to proceed.


## ※ここでDICOM dataを DICOM/original に入れる！\
フォルダ名が研究の画像IDになるように名前をつけておく

## original DICOM データの匿名化+研究ID付与

In [4]:
#DICOMメタデータの匿名化+研究IDの付与
path2 = f"{project}/DICOM/original/"
!python ~/git/yn-scripts/change_patientIDname.py {path2}

Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/00153.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/00635.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/00621.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/00147.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/01259.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/01271.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/01517.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/00609.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702/08/01503.dcm to NIID20240702
Updated PatientName and PatientID in NIIDtest/DICOM/original/NIID20240702

## DICOM データのソート

In [5]:
!~/git/yn-scripts/yn-batch-heudiconv/bh02_sort_dicom_yn.sh {project}

Replacing spaces with underscores in directory and file names...
Sorting DICOM files...
Processing directory: NIID20240702
Failed to process NIID20240702/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Failed to process NIID20240702/._.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Sorted NIID20240702/08/00153.dcm to ../sorted/NIID20240702/901_DTI_opt_32_MB2/00153.dcm
Sorted NIID20240702/08/00635.dcm to ../sorted/NIID20240702/901_DTI_opt_32_MB2/00635.dcm
Sorted NIID20240702/08/00621.dcm to ../sorted/NIID20240702/901_DTI_opt_32_MB2/00621.dcm
Sorted NIID20240702/08/00147.dcm to ../sorted/NIID20240702/901_DTI_opt_32_MB2/00147.dcm
Sorted NIID20240702/08/01259.dcm to ../sorted/NIID20240702/901_DTI_opt_32_MB2/01259.dcm
Sorted NIID20240702/08/01271.dcm to ../sorted/NIID20240702/901_DTI_opt_32_MB2/01271.d

## 被験者リストの作成

In [6]:
#ディレクトリ命名パターンに基づいて被験者リストを生成
!bh03_make_subjlist.sh {project} {pattern}

Subject list has been created: tmp/subjlist_NIIDtest.tsv
Content of the subject list:
directory	subject_ID	session
{subject}	NIID20240702	01

Please verify the subject list before proceeding with BIDS conversion.


## Heuristicファイルの生成 (α版)

In [7]:
!bh04_make_heuristic.sh {project}

Scanning directory structure in NIIDtest/DICOM/sorted...
Auto-detected Siemens scanner (default)
Analyzing sequence directories...
  Analyzed: 201_T1FFE3D_SAG2 -> Type: unknown, Direction: none, SBRef: no
  Analyzed: 901_DTI_opt_32_MB2 -> Type: dwi, Direction: none, SBRef: no
Generating heuristic.py file...
/Users/yn_mbp/git/batch-heudiconv/bh04_make_heuristic.sh: line 328: s: command not found
Heuristic file created: NIIDtest/code/heuristic_NIIDtest.py

Please review and edit the file if necessary.
The generated file contains:
1. Anatomical scan definitions
2. Functional scans
3. Diffusion weighted imaging
4. Fieldmaps
5. Appropriate handling of phase encoding directions if detected


### heuristicファイルを確認・修正する

In [12]:
!code {project}/code/heuristic_{project}.py

In [15]:
'''
### nifti filenames ###

    # T1
    t1w = create_key('sub-{subject}/anat/sub-{subject}_run-{item:02d}_T1w')

    # DTI
    dwi = create_key('sub-{subject}/dwi/sub-{subject}_run-{item:02d}_dwi')

    info = {t1w: [], dwi: []}


 ### extract keywords from sorted DICOM series-based sub-directories and dimensions ###
    # T1w
    if 'T1FFE' in s.dcm_dir_name:
        info[t1w].append(s.series_id)
    # DWI
    if '901_DTI_opt_32_MB2' in s.dcm_dir_name:
        info[dwi].append(s.series_id)

'''

"\n### nifti filenames ###\n\n    # T1\n    t1w = create_key('sub-{subject}/anat/sub-{subject}_run-{item:02d}_T1w')\n\n    # DTI\n    dwi = create_key('sub-{subject}/dwi/sub-{subject}_run-{item:02d}_dwi')\n\n    info = {t1w: [], dwi: []}\n\n\n ### extract keywords from sorted DICOM series-based sub-directories and dimensions ###\n    # T1w\n    if 'T1FFE' in s.dcm_dir_name:\n        info[t1w].append(s.series_id)\n    # DWI\n    if '901_DTI_opt_32_MB2' in s.dcm_dir_name:\n        info[dwi].append(s.series_id)\n\n"

## BIDSへの変換

In [13]:
!bh05_make_bids.sh {project}

Starting BIDS conversion...
Processing: Subject=NIID20240702 Session=01
INFO: Running heudiconv version 1.1.3 latest Unknown
INFO: Need to process 1 study sessions
INFO: PROCESSING STARTS: {'subject': 'NIID20240702', 'outdir': '/Volumes/YN_Research/NIIDtest/bids/rawdata/', 'session': '01'}
INFO: Processing 2048 dicoms
INFO: Will not reuse existing conversion table files because heuristic has changed
INFO: Analyzing 2048 dicoms
INFO: Generated sequence info for 2 studies with 2048 entries total
INFO: Doing conversion using dcm2niix
INFO: Converting /Volumes/YN_Research/NIIDtest/bids/rawdata/sub-NIID20240702/anat/sub-NIID20240702_run-01_T1w (200 DICOMs) -> /Volumes/YN_Research/NIIDtest/bids/rawdata/sub-NIID20240702/anat . Converter: dcm2niix . Output types: ('nii.gz',)
250517-21:07:47,588 nipype.workflow INFO:
	 [Node] Setting-up "convert" in "/private/var/folders/l7/cr26cmms1j5dkct3b2vz762c0000gn/T/dcm2niix_x9hyllu/convert".
INFO: [Node] Setting-up "convert" in "/private/var/folders/l7/

In [14]:
#ディレクトリ構造の確認
!tree {project}/bids/rawdata

NIIDtest/bids/rawdata
├── CHANGES
├── README
├── dataset_description.json
├── participants.json
├── participants.tsv
├── scans.json
└── sub-NIID20240702
    ├── anat
    │   ├── sub-NIID20240702_run-01_T1w.json
    │   └── sub-NIID20240702_run-01_T1w.nii.gz
    ├── dwi
    │   ├── sub-NIID20240702_run-01_dwi.bval
    │   ├── sub-NIID20240702_run-01_dwi.bvec
    │   ├── sub-NIID20240702_run-01_dwi.json
    │   └── sub-NIID20240702_run-01_dwi.nii.gz
    └── sub-NIID20240702_scans.tsv

4 directories, 13 files


## BIDS validator で確認する

https://bids-standard.github.io/bids-validator \
上記リンクにアクセスし、bids/raw を選択 \
Warningはいっぱい出ても、エラーが出なければOK